# Description
#### author Houcmeddine Othman, September 2019, [My Github page](https://github.com/hothman) 

The codes in this notebook are written to generate the structure of the database accroding to the ERD including the columns for primary keys and foreign keys. The output are 7 tables in csv format. 

![title](diagram_PM.png)

### requirements:
* Python 3
* Pandas, numpy, pubmed_parser
---
## Credits
### Without the contribution of these people, I would end up with ugly empty output files and big ERROR message. 

#### Manual Curation (in alphabetical order) 
Ayoub Ksouri, Chaimae SAMTAL, Chiamaka Jessica Okeke, Fouzia Radouani, Haifa Jmal, Kais Ghedira, Lyndon Zass, Melek Chaouch, Olivier, Reem Sallam, Rym Kefi, Samah Ahmed, Samar kamel Kassem, Yosr Hamdi.

#### Data mining and wrangling
Jorge da Rocha and Lyndon Zass

## Completing P-value to SNP and Star tables

In [431]:
import pandas as pd 
import warnings

# read tables 
snp = pd.read_csv("../data/snp_raw.csv")
star = pd.read_csv("../data/star_raw.csv")
curation = pd.read_csv("../data/clean_curation.csv")

# Some variants were discarded in in the curation table, we delete them also from snp and star
ids_in_curation = list( curation.id_in_source )
snp = snp[snp.id_in_source.isin(ids_in_curation)]
star = star[star.id_in_source.isin(ids_in_curation)]

# left joining curation data to snp table
snp = pd.merge(snp, curation[["id_in_source", "P-value"]] , on="id_in_source")

# left joining curation data to star allele table
star = pd.merge(star, curation[["id_in_source", "P-value"]] , on="id_in_source")

# some of the variants are going to be deleted 
var_to_delete = ['1444665865','1444665876', '1444665886', '1449270328', 
 '1449270311', '1449270340', '1184509852','1184509863', '1448105757']

for varID in var_to_delete: 
    snp = snp[snp.id_in_source != int(varID)]   # panas reads var as integer 
    star = star[star.id_in_source != int(varID)]
    curation = curation[curation.id_in_source != int(varID)]

#generating primary keys for snp table 
len_snp = len(snp)
ID_snp = [ "SNP"+str(i) for i in range(1, len_snp+1) ]
snp["ID_snp"] = ID_snp

#generating primary keys for star table 
len_star = len(star)
ID_star = [ "STAR"+str(i) for i in range(1, len_star+1) ]
star["ID_star"] = ID_star

snp

,rs_id,gene_name,drug_name,phenotype,allele,reference_id,source,id_in_source,P-value,ID_snp
0,rs28371685,CYP2C9,warfarin,Allele T is associated with decreased dose of ...,T,20072124,PharmGKB,608431789,< 0.001,SNP1
1,rs28371686,CYP2C9,warfarin,Allele G is associated with decreased dose of ...,G,20072124,PharmGKB,608431793,< 0.001,SNP2
2,rs7900194,CYP2C9,warfarin,Allele A is associated with decreased dose of ...,A,20072124,PharmGKB,608431781,0.023,SNP3
3,rs9332131,CYP2C9,warfarin,Genotype DEL is associated with decreased dose...,DEL,20072124,PharmGKB,608431785,< 0.001,SNP4
4,rs339097,CALU,warfarin,Allele G is associated with increased dose of ...,G,20200517,PharmGKB,637879876,0.03,SNP5
...,...,...,...,...,...,...,...,...,...,...
362,rs1902023,UGT2B15,acetaminophen,Genotype AA is associated with increased metab...,AA,28663312,PharmGKB,1448639952,ambiguous,SNP363
363,CYP3A5,CYP3A5 (PA131),acetaminophen,Allele C is not associated with metabolism of ...,C,28663312,PharmGKB,1448639975,> 0.05,SNP364
364,rs8330,UGT1A,acetaminophen,Allele C is not associated with metabolism of ...,C,28663312,PharmGKB,1448639969,> 0.05,SNP365
365,rs5333,EDNRA,prednisone,Genotypes CC + CT is associated with increased...,CC + CT,30672385,PharmGKB,1450367980,0.016,SNP366


## generating tables ethnicity_country for snp and star 


In [432]:
regions= list( curation.groupby('region').count().index )
len_regions = len(regions)
ID_region = [ "REGION"+str(i) for i in range(1, len_regions+1) ]
region_table = pd.DataFrame({"id_region":ID_region, "region":regions })

region_table

,id_region,region
0,REGION1,African American/Afro-Caribbean
1,REGION2,Mixed Population containing african descendant...
2,REGION3,North African
3,REGION4,Sub-Saharan African


## generating table for reference

In [433]:
reference = pd.read_csv("../data/referance_raw.csv")
len_reference = len(reference)
ID_reference = [ "REF"+str(i) for i in range(1, len_reference+1) ]
reference["ref_id"]=ID_reference
reference.rename(columns={'external_id': "reference_id"}, inplace=True)
reference.set_index("ref_id", inplace=True)

reference.to_csv("../data/csv4SQL/Study.csv", index=False)


reference.head()

,reference_id,type,year,title
ref_id,,,,
REF1,24024897,PMID,2013,RRM1 and RRM2 pharmacogenetics: association wi...
REF2,23173844,PMID,2012,PXR and CAR single nucleotide polymorphisms in...
REF3,27045425,PMID,2016,CYP2B6 genotype-based efavirenz dose recommend...
REF4,15094935,PMID,2004,CYP2C9 genetic polymorphisms and warfarin.
REF5,23158458,PMID,2013,Multiple regulatory variants modulate expressi...


## Generating pharmacogenes table 


In [434]:
pharmacogenes = pd.read_csv("../data/genes_prot_chromosome_function.csv")
len_pharmacogenes = len(pharmacogenes)
ID_pharmacogenes = [ "PHARGENE"+str(i) for i in range(1, len_pharmacogenes+1) ]
pharmacogenes["ID_pharmacogenes"] = ID_pharmacogenes 
pharmacogenes.set_index("ID_pharmacogenes", inplace=True)

pharmacogenes.to_csv("../data/csv4SQL/pharmacogenes.csv", index=False)

pharmacogenes.head()

,Chromosome,gene_name,Uniprot_ID,Function
ID_pharmacogenes,,,,
PHARGENE1,7,ABCB1,P08183,Translocates drugs and phospholipids across th...
PHARGENE2,2,ABCB11,O95342,Catalyzes the secretion of conjugated bile sal...
PHARGENE3,10,ABCC2,Q92887,Mediates hepatobiliary excretion of numerous o...
PHARGENE4,4,ABCG2,Q9UNQ0,High-capacity urate exporter functioning in bo...
PHARGENE5,17,ACE,P12821,Converts angiotensin I to angiotensin II by re...


## Generating drug table

In [442]:
drugs = pd.read_csv("../data/drugs.csv")
len_drugs = len(drugs)
ID_drugs = [ "DRUG"+str(i) for i in range(1, len_drugs+1) ]
drugs["ID_drugs"] = ID_drugs 
drugs.set_index("ID_drugs", inplace=True)
drugs.rename(columns={'Drug name': "drug_name"}, inplace=True)

drugs.to_csv("../data/csv4SQL/drug.csv", index=False)

drugs.head()


,drug_name,ID Drug bank,state,Indication,IUPAC_name
ID_drugs,,,,,
DRUG1,warfarin,DB00682,Approved,Prophylaxis and treatment of venous thromboemb...,4-hydroxy-3-(3-oxo-1-phenylbutyl)-2H-chromen-2...
DRUG2,haloperidol,DB00502,Approved,Haloperidol is indicated for a number of condi...,4-[4-(4-chlorophenyl)-4-hydroxypiperidin-1-yl]...
DRUG3,clozapine,DB00363,Approved,For use in patients with treatment-resistant s...,8-Chloro-11-(4-methylpiperazin-1-yl)-5H-dibenz...
DRUG4,mephenytoin,DB00532,"Approved, Investigational, Withdrawn",For the treatment of refractory partial epilepsy,"5-ethyl-3-methyl-5-phenylimidazolidine-2,4-dione"
DRUG5,efavirenz,DB00625,"Approved, Investigational",(4S)-6-chloro-4-(2-cyclopropylethynyl)-4-(trif...,"(4S)-6-Chloro-4-(cyclopropylethynyl)-1,4-dihyd..."


# Arranging secondary keys for SQL implementation

### SNP table

In [436]:
def create_col_from_index(df_on_left, df_On_right, col_to_merge, tag ):
    mydf = df_On_right.copy()
    index_col = mydf.index
    mydf["index_col"] = index_col    
    mydf=mydf.filter(items=['index_col', col_to_merge ] )  
    merged= pd.merge(df_on_left, mydf, on=col_to_merge).drop(columns=[col_to_merge])
    return merged.rename(columns={'index_col': tag} )


# add gene id
snp = create_col_from_index(snp, pharmacogenes , "gene_name", "gene_id" )

# add drug id, corrected later aftergenerating the whole able
snp = create_col_from_index(snp, drugs , "drug_name", "drug_id" )

# add reference id 
snp = create_col_from_index(snp, reference , "reference_id", "reference_id" ) 


# arranging columns 
snp.rename(columns={'ID_snp': "id"}, inplace=True )

snp= snp.filter(items=['id', "rs_id", "drug_id", 
                  "allele", "phenotype", 
                  "reference_id", "P-value", 
                 "source", "id_in_source"])

snp.to_csv("../data/csv4SQL/snp_allele.csv", index=False)


### Star allele table

In [437]:
# add gene id
star= create_col_from_index(star, pharmacogenes , "gene_name", "gene_id" )
# add drug id, corrected later aftergenerating the whole able
star = create_col_from_index(star, drugs , "drug_name", "drug_id" )
# add reference id 
star = create_col_from_index(star, reference , "reference_id", "reference_id" ) 
# arranging columns 
star.rename(columns={'ID_star': "id"}, inplace=True )

star= star.filter(items=['id', "star_annotation", "drug_id", 
                  "allele", "phenotype", 
                  "reference_id", "P-value", 
                 "source", "id_in_source"])


star.to_csv("../data/csv4SQL/star_allele.csv", index=False)


### snp by country table

In [438]:
merged_countries = pd.merge(snp, curation, on="id_in_source")
merged_countries_snp = merged_countries.filter(items=["id", "region", "Country_of_Participants"]) 
merged_countries_snp.rename(columns={'id': "snp_id"}, inplace=True )
len_merged_countries_snp = len(merged_countries_snp)
ID_countries_snp = [ "SNPREGION"+str(i) for i in range(1, len_merged_countries_snp+1) ]
merged_countries_snp["id"]=ID_countries_snp
merged_countries_snp = merged_countries_snp.filter(items=["id", "snp_id", "region", "Country_of_Participants"])

# output table 
merged_countries_snp.to_csv("../data/csv4SQL/snp_country.csv", index=False)

merged_countries_snp.head()

,id,snp_id,region,Country_of_Participants
0,SNPREGION1,SNP1,African American/Afro-Caribbean,USA
1,SNPREGION2,SNP1,African American/Afro-Caribbean,USA
2,SNPREGION3,SNP2,African American/Afro-Caribbean,USA
3,SNPREGION4,SNP2,African American/Afro-Caribbean,USA
4,SNPREGION5,SNP3,African American/Afro-Caribbean,USA


### star by country 

In [439]:
merged_countries = pd.merge(star, curation, on="id_in_source")
merged_countries_star = merged_countries.filter(items=["id", "region", "Country_of_Participants"]) 
merged_countries_star.rename(columns={'id': "star_id"}, inplace=True )
len_merged_countries_star = len(merged_countries_star)
ID_countries_star = [ "STARREGION"+str(i) for i in range(1, len_merged_countries_star+1) ]
merged_countries_star["id"]=ID_countries_star
merged_countries_star = merged_countries_star.filter(items=["id", "star_id", "region", "Country_of_Participants"])

# output the table
merged_countries_star.to_csv("../data/csv4SQL/star_allele_country.csv", index=False)

merged_countries_star.head()

,id,star_id,region,Country_of_Participants
0,STARREGION1,STAR1,Sub-Saharan African,Ethiopia
1,STARREGION2,STAR2,African American/Afro-Caribbean,USA
2,STARREGION3,STAR3,African American/Afro-Caribbean,USA
3,STARREGION4,STAR196,African American/Afro-Caribbean,USA
4,STARREGION5,STAR197,African American/Afro-Caribbean,USA
